In [1]:
print("Done")

Done


In [2]:
%matplotlib inline
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [3]:
# set up file names and pathes
dataDir='.'
dataType='val2017'
algName = 'fakecap'
annFile='%s/annotations/captions_%s.json'%(dataDir,dataType)
print(annFile)
subtypes=['results', 'evalImgs', 'eval']
[resFile, evalImgsFile, evalFile]= \
['%s/results/captions_%s_%s_%s.json'%(dataDir,dataType,algName,subtype) for subtype in subtypes]
print(resFile, evalImgsFile, evalFile)
# download Stanford models
#./get_stanford_models.sh

./annotations/captions_val2017.json
./results/captions_val2017_fakecap_results.json ./results/captions_val2017_fakecap_evalImgs.json ./results/captions_val2017_fakecap_eval.json


In [4]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
0:00:00.074014
creating index...
index created!
Loading and preparing results...     
DONE (t=0.01s)
creating index...
index created!


In [5]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 50543, 'reflen': 50023, 'guess': [50543, 45554, 40565, 35576], 'correct': [23190, 5560, 632, 127]}
ratio: 1.010395218199608
Bleu_1: 0.459
Bleu_2: 0.237
Bleu_3: 0.096
Bleu_4: 0.042
computing METEOR score...
METEOR: 0.104
computing Rouge score...
ROUGE_L: 0.333
computing CIDEr score...
CIDEr: 0.057
computing SPICE score...
SPICE: 0.024


In [6]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print('%s: %.3f'%(metric, score))

Bleu_1: 0.459
Bleu_2: 0.237
Bleu_3: 0.096
Bleu_4: 0.042
METEOR: 0.104
ROUGE_L: 0.333
CIDEr: 0.057
SPICE: 0.024


In [7]:
# demo how to use evalImgs to retrieve low score result
evals = [eva for eva in cocoEval.evalImgs if eva['CIDEr']<30]
print('ground truth captions')
imgId = evals[0]['image_id']
annIds = coco.getAnnIds(imgIds=imgId)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

print('\n')
print('generated caption (CIDEr score %0.1f)'%(evals[0]['CIDEr']))
annIds = cocoRes.getAnnIds(imgIds=imgId)
anns = cocoRes.loadAnns(annIds)
coco.showAnns(anns)

img = coco.loadImgs(imgId)[0]
I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
plt.imshow(I)
plt.axis('off')
plt.show()

ground truth captions
A man is in a kitchen making pizzas.
Man in apron standing on front of oven with pans and bakeware
A baker is working in the kitchen rolling dough.
A person standing by a stove in a kitchen.
A table with pies being made and a person standing near a wall with pots and pans hanging on the wall.


generated caption (CIDEr score 0.2)
a man is a man in a white .


FileNotFoundError: [Errno 2] No such file or directory: './images/val2017/000000397133.jpg'

In [ ]:
# plot score histogram
ciderScores = [eva['CIDEr'] for eva in cocoEval.evalImgs]
plt.hist(ciderScores)
plt.title('Histogram of CIDEr Scores', fontsize=20)
plt.xlabel('CIDEr score', fontsize=20)
plt.ylabel('result counts', fontsize=20)
plt.show()

In [ ]:
# save evaluation results to ./results folder
json.dump(cocoEval.evalImgs, open(evalImgsFile, 'w'))
json.dump(cocoEval.eval,     open(evalFile, 'w'))